In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

def crawl(url, domain, visited=None):
    if visited is None:
        visited = set()

    # 이미 방문한 URL은 다시 크롤링하지 않도록 체크
    if url in visited:
        return
    visited.add(url)

    print(f"크롤링 중: {url}")

    try:
        # HTTP GET 요청 보내기 (필요시 headers 등을 추가할 수 있음)
        response = requests.get(url)
        if response.status_code != 200:
            print(f"페이지 요청 실패: {url} (상태 코드: {response.status_code})")
            return

        # HTML 소스 코드 가져오기
        html_content = response.text

        # BeautifulSoup으로 HTML 파싱하기
        soup = BeautifulSoup(html_content, 'html.parser')

        # 모든 <a> 태그의 href 속성 추출 (내부 링크 위주로 처리)
        for a_tag in soup.find_all('a'):
            href = a_tag.get('href')
            if not href:
                continue

            # 상대 경로인 경우, 절대 URL로 변환
            full_url = urljoin(url, href)
            parsed_url = urlparse(full_url) 

            # 동일 도메인 내 링크만 처리 
            if parsed_url.netloc == domain:
                visited.add(full_url)
                # 필요에 따라 재귀적으로 크롤링할 수 있음 (현재는 주석 처리)
                # crawl(full_url, visited)
        return list(visited)
    except Exception as e:
        print(f"오류 발생: {e}")


In [13]:
URLS = crawl("https://sociology.jnu.ac.kr/sociology/8677/subview.do","sociology.jnu.ac.kr")

for url in URLS:
    if "rss" in url:
        rss_url = url
        break 

rss_url = rss_url[ :len(rss_url) - 2] 
"""노가다"""
number = "708"################################################
rss_url = rss_url + number
rss_url


크롤링 중: https://sociology.jnu.ac.kr/sociology/8677/subview.do


'https://sociology.jnu.ac.kr/bbs/sociology/1001/rssList.do?row=708'

In [14]:
import feedparser
import pandas as pd

feed = feedparser.parse(rss_url)

"""노가다다"""
department = "사회학과"######################################

# 데이터를 리스트로 저장
entries = []
for entry in feed.entries:
    entries.append({
        "학부": department,
        "제목": entry.title,
        "링크": entry.link,
        "내용": entry.summary,
        "작성일": entry.published
    })

# 데이터프레임 변환
df = pd.DataFrame(entries)

# CSV 파일로 저장
df.to_csv(f"{department} rss_feed.csv", index=False, encoding="utf-8-sig")

# JSON 파일로 저장
df.to_json(f"{department} rss_feed.json", orient="records", force_ascii=False)

print("CSV 및 JSON 파일 저장 완료!")

CSV 및 JSON 파일 저장 완료!
